In [3]:
%env kubectl=/usr/bin/kubectl
%env istio=/Users/gsjurseth/istio-1.0.2
%env istioctl=/Users/gsjurseth/istio-1.0.2/bin/istioctl
%env ingress=35.226.214.129
%env demo=/Users/gsjurseth/dev/fazio/microservices-demo

env: kubectl=/usr/bin/kubectl
env: istio=/Users/gsjurseth/istio-1.0.2
env: istioctl=/Users/gsjurseth/istio-1.0.2/bin/istioctl
env: ingress=35.226.214.129
env: demo=/Users/gsjurseth/dev/fazio/microservices-demo


---
## Fazio Retail Shop UI

http://35.226.214.129/


Let's look at the running pods

In [7]:
!$kubectl get pods

NAME                                     READY   STATUS             RESTARTS   AGE
adservice-67884759d9-7zdv2               2/2     Running            0          6m
cartservice-7bfdb5c8c4-rgnn5             1/2     CrashLoopBackOff   4          3m
checkoutservice-7dcb8b7b76-h7vk2         2/2     Running            0          3m
currencyservice-646d587bc7-h9cpx         2/2     Running            0          2m
emailservice-5b6f4f44bd-b9xr2            2/2     Running            0          2m
frontend-blue-6c56684d8-lb2wf            2/2     Running            0          2m
frontend-gray-74574c4564-vvb2z           2/2     Running            0          2m
loadgenerator-6cd67f4d67-4bl9r           0/2     Init:0/2           0          2m
paymentservice-6565588d84-xcdjv          2/2     Running            0          2m
productcatalogservice-86d7c6744f-wnqql   2/2     Running            0          2m
recommendationservice-5c5fd5cf7f-cwvhf   2/2     Running            0          2m
redis-cart-6c5b

---
## Architectural Diagram

![architecture](https://raw.githubusercontent.com/theganyo/microservices-demo/fazio-playground/img/architecture-diagram.jpeg)

---
## I've heard that blue is hip color!

But is it really? Let's canary test a blue front-end. We'll route 20% of traffic to the blue version.

In [5]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: frontend-ingress
spec:
  gateways:
  - frontend-gateway
  hosts:
  - '*'
  http:
  - route:
    - destination:
        host: frontend
        subset: gray
      weight: 66
    - destination:
        host: frontend
        subset: blue
      weight: 34

EOF

virtualservice.networking.istio.io/frontend-ingress configured


---
## It turns out, customers prefer grey.

Let's revert that change.

In [6]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: frontend-ingress
spec:
  gateways:
  - frontend-gateway
  hosts:
  - '*'
  http:
  - route:
    - destination:
        host: frontend
        subset: gray
      weight: 100

EOF

virtualservice.networking.istio.io/frontend-ingress configured


---
## Jaeger UI

Run in console:

    kubectl port-forward -n istio-system $(kubectl get pod -n istio-system -l app=jaeger -o jsonpath='{.items[0].metadata.name}') 16686:16686 &
    
http://penguin.linux.test:16686/search?service=emailservice

---
## Grafana UI

Run in console:

    kubectl -n istio-system port-forward $(kubectl -n istio-system get pod -l app=grafana -o jsonpath='{.items[0].metadata.name}') 3000:3000 &
    
http://penguin.linux.test:3000/d/3/pilot-dashboard